In [21]:
from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np

In [22]:
# generate quantum circuits representing each bit in the message
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [23]:
def measure_message(message, bases):
    
    measurements = []
    
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    
    return measurements

In [24]:
# Removes bit measurement that we cannot guarantee match
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
   
    return good_bits

In [25]:
#Sample bits from selction to compare
def sample_bits(bits, selection):
    sample = []
    
    for i in selection:
        i = np.mod(i, len(bits)) # make sure i is in lisy range
        sample.append(bits.pop(i))
    return sample

In [29]:
np.random.seed(seed=0)
n = 100

# Alice generates bits
alice_bits = randint(2, size=n)

# generate random bases to encode message in
alice_bases = randint(2, size=n)
message = encode_message(alice_bits, alice_bases)

# Bob measures using randomly generated bases
bob_bases = randint(2, size=n)
bob_results = measure_message(message, bob_bases)

## Alice and Bob share the bases they used to measure (over Eve's channel) and remove garbage 
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)

## Step 5
sample_size = 15
bit_selection = randint(n, size=sample_size)

bob_sample = sample_bits(bob_key, bit_selection)
print("  Bob's sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("Alice's sample = "+ str(alice_sample))

print("Samples match: "+ str(bob_sample == alice_sample))

  Bob's sample = [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Alice's sample = [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Samples match: True


In [30]:
print(bob_key)
print(alice_key)
print("key length = %i" % len(alice_key))

[1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]
key length = 33


In [31]:
np.random.seed(seed=3)
n = 100

# Alice generates message and encodes it
alice_bits = randint(2, size=n)
alice_bases = randint(2, size=n)
message = encode_message(alice_bits, alice_bases)

# Eve intercepts
eve_bases = randint(2, size=n)
intercepted_message = measure_message(message, eve_bases)

# Bob measures message after interception
bob_bases = randint(2, size=n)
bob_results = measure_message(message, bob_bases)

bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)

# Alice and Bob sample their results and compare
sample_size = 15 # Increasing the sample size reduces the likelihood of Eve going undetected
bit_selection = randint(n, size=sample_size)
bob_sample = sample_bits(bob_key, bit_selection)
alice_sample = sample_bits(alice_key, bit_selection)

if bob_sample != alice_sample:
    print("Eve's interference was detected.")
else:
    print("Eve was undetected")

Eve's interference was detected.
